# 4 урок. 3 задание.

Разобранными на лекции методами список того, что можно применять к метрикам-отношениям, не ограничивается. Есть огромное количество полезных материалов на эту тему. Начнем с материалов Никиты Маршалкина. К слову, вот его интервью, там тоже много интересного. 

Относительно недавно (в 2018-м году) исследователи из Яндекса разработали классный метод анализа тестов над метриками-отношениями (прямо как у нас) вида  𝑥𝑦.

<b>Идея метода заключается в следующем:</b>

Вместо того, чтобы заталкивать в тест «поюзерные» CTR, можно сконструировать другую метрику и анализировать ее, но при этом гарантируется (в отличие от сглаженного CTR), что если тест на этой другой метрике «прокрасится» и увидит изменения, значит изменения есть и в метрике исходной (то есть в лайках на пользователя и в пользовательских CTR

При этом метод сам по себе очень прост. Что это за метрика такая?

- Считаем общий CTR в контрольной группе  𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠) 
- Посчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠 
- После чего сравним  t-тестом отличия в группах по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠  

Метод простой, гарантируется, что при приличном размере выборки (как у нас — подойдет) можно бесплатно увеличить чувствительность вашей метрики (или, по крайней мере, не сделать хуже). Как по мне, это ОЧЕНЬ круто.

## Задача:

1. Проанализируйте тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше по сравнению с обычным CTR?
2. Проанализируйте тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше по сравнению с обычным CTR?
3. Данные берём в том же диапазоне, в котором проводился АБ-тест.

При выполнении задания важно обосновать и аргументировать ваши выводы.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pandahouse
from scipy import stats
from matplotlib import pyplot as plt

## Подключение к БД и подгрузка данных

In [2]:
connection = {'host': 'https://clickhouse.lab.karpov.courses',
'database':'simulator_20240720',
'user':'student',
'password':'dpo_python_2020'
}

In [7]:
q = '''
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions
WHERE toDate(time) between '2024-06-28' and '2024-07-04'
GROUP BY exp_group, user_id
'''

In [8]:
df = pandahouse.read_clickhouse(q, connection=connection)

In [9]:
df.head(10)

,exp_group,user_id,likes,views,ctr
0,3,115383,12,44,0.272727
1,1,18392,7,32,0.218750
2,3,123580,2,11,0.181818
3,2,131473,9,108,0.083333
4,4,107323,5,31,0.161290
5,2,32420,13,44,0.295455
6,2,24239,40,90,0.444444
7,0,30319,25,114,0.219298
8,1,27443,22,109,0.201835
9,3,118359,7,38,0.184211


### Задача 1.

Проанализируйте тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше по сравнению с обычным CTR?

In [37]:
def ttest_upgrade(control, test):
    control_ctr = control.likes.sum() / control.views.sum()
    
    lin_likes_c = control.likes - control_ctr*control.views
    lin_likes_t = test.likes - control_ctr*test.views
    
    return stats.ttest_ind(lin_likes_c,
                           lin_likes_t,
                           equal_var = False)

In [38]:
ttest_upgrade(df[df['exp_group'] == 0], df[df['exp_group'] == 3])

Ttest_indResult(statistic=-16.18623003293285, pvalue=1.4918137745324434e-58)

In [41]:
# Для сравнения выведем результаты т-теста по СТР
stats.ttest_ind(df[df['exp_group'] == 0].ctr, df[df['exp_group'] == 3].ctr, equal_var = False)

Ttest_indResult(statistic=-13.935320516755823, pvalue=6.216047483057877e-44)

### Вывод
Как видно из тестов, апгрейднутый тест дает p-value меньше, чем стандарный ттест по стр.

### Задача 2
Проанализируйте тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше по сравнению с обычным CTR?

In [42]:
ttest_upgrade(df[df['exp_group'] == 1], df[df['exp_group'] == 2])

Ttest_indResult(statistic=5.936377101934478, pvalue=2.9805064038668383e-09)

In [43]:
# Выведем значение ттеста стандартным способом
stats.ttest_ind(df[df['exp_group'] == 1].ctr, df[df['exp_group'] == 2].ctr, equal_var = False)

Ttest_indResult(statistic=0.40514919131127564, pvalue=0.685373331140751)

### Вывод
Результат в апгрейднутом тесте действительно изменился и более того, при стандартном замере поюзерного стр мы получили результат, подтверждающий нулевую гипотезу, а при апгрейднутом тесте результат, отклоняющий нулевую гипотезу

# Общий вывод

По результатам теста можно сказать, что новый алгоритм действительно улучшает результаты тестирования, особенно, это видно по результатам второго теста (1 и 2 группы)